In [2]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [14, 8]
import motmetrics as mm
from motmetrics.distances import iou_matrix
import pandas as pd

sys.path.append('../..')
from pytracking.utils.load_text import load_text
from pytracking.evaluation import Tracker, get_dataset, trackerlist
from pprint import pprint

# trackers = [Tracker('atom','default')]
trackers = [Tracker('tomp','tomp50')]
# dataset = get_dataset('got10k_val')
# dataset = get_dataset('yt2019_sub')
# dataset = get_dataset('got10k_probio')
# dataset = get_dataset('got10k_probio_valid')
d1 = get_dataset('yt_pb_train')
d2 = get_dataset('yt_pb_valid')

def motMetricsEnhancedCalculator(gt, t):
  # import required packages
  
  acc = mm.MOTAccumulator(auto_id=True)

  # Max frame number maybe different for gt and t files
  for frame in range(int(gt[:,0].max())):
    frame += 1 # detection and frame numbers begin at 1

    # select id, x, y, width, height for current frame
    # required format for distance calculation is X, Y, Width, Height \
    # We already have this format
    gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
    t_dets = t[t[:,0]==frame,1:6] # select all detections in t

    C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                max_iou=0.5) # format: gt, t

    # Call update once for per frame.
    # format: gt object ids, t object ids, distance
    acc.update(gt_dets[:,0].astype('int').tolist(), \
              t_dets[:,0].astype('int').tolist(), C)
    
  return acc

def one_track(trk):
    names = []
    accs = []

    for seq_id, seq in enumerate(dataset):
        base_results_path = '{}/{}'.format(trk.results_dir, seq.name)
        gt = []
        pred = []
        for (obj_id, obj_gt) in list(seq.ground_truth_rect.items()):
            results_path = '{}_{}.txt'.format(base_results_path,obj_id)
            if os.path.isfile(results_path):
                pred_bb = torch.tensor(load_text(str(results_path), delimiter=('\t', ','), dtype=np.float64))
                obj_gt = torch.tensor(obj_gt,dtype=torch.float64)
            else:
                print('Result not found. {}'.format(results_path))
                continue
            for frame_id, b in enumerate(obj_gt):
                if b[0] == -1: continue
                gt.append([frame_id+1,obj_id,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
            for frame_id, b in enumerate(pred_bb):
                if b[0] == -1: continue
                pred.append([frame_id+1,obj_id,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
        # if seq.name in ['0043f083b5','0044fa5fba']:
        #     slices = slice(2,6)
        #     print(f'----------------{seq.name}-------------')
        #     cur_id = 0
        #     for i,j in zip(gt,pred):
        #         if i[1] != cur_id:
        #             print(f'----------------{i[1]}-------------')
        #             cur_id = i[1]
        #         a = [f'{int(ii)-int(jj):>3}' for ii,jj in zip(i[slices],j[slices])]
        #         bi = [int(ii) for ii in i[slices]]
        #         bj = [int(jj) for jj in j[slices]]
        #         c = iou_matrix([i[slices]],[j[slices]])[0][0]
        #         print(f'{c:>5.3f}',a,bi,bj)
        if len(gt) == 0: continue
        accs.append(motMetricsEnhancedCalculator(np.array(gt, dtype=np.float64), np.array(pred, dtype=np.float64)))
        names.append(seq.name)


    mh = mm.metrics.create()
    summary = mh.compute_many(accs, metrics=['mota', 'motp', 'num_frames', 'num_objects' , 'idf1', 'idp', 'idr', \
                                        'recall', 'precision',  \
                                        'mostly_tracked', 'partially_tracked', \
                                        'mostly_lost', 'num_false_positives', \
                                        'num_misses', 'num_switches', \
                                        'num_fragmentations'
                                    ], \
                        generate_overall=True,
                        names=names)

    # strsummary = mm.io.render_summary(
    #     summary,
    #     #formatters={'mota' : '{:.2%}'.format},
    #     namemap={'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
    #              'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
    #             'precision': 'Prcn',  \
    #             'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
    #             'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
    #             'num_misses': 'FN', 'num_switches' : 'IDsw', \
    #             'num_fragmentations' : 'FM'
    #             }
    # )

    df =  pd.DataFrame.from_dict(summary)
    namemap = {'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
                 'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', 'precision': 'Prcn',  \
                'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
                'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
                'num_misses': 'FN', 'num_switches' : 'IDsw', \
                'num_fragmentations' : 'FM', 'num_frames': 'Frames'
                }
    df.rename(columns=namemap, inplace=True)
    return df

def one_track_single(trk):
    names = []
    accs = []
    
    for seq_id, seq in enumerate(dataset):
        results_path = '{}/{}.txt'.format(trk.results_dir, seq.name)
        obj_gt = seq.ground_truth_rect
        gt = []
        pred = []
        
        def compare():
            slices = slice(2,6)
            print(f'----------------{seq.name}-------------')
            cur_id = 0
            for i,j in zip(gt,pred):
                if i[1] != cur_id:
                    print(f'----------------{i[1]}-------------')
                    cur_id = i[1]
                a = [f'{int(ii)-int(jj):>3}' for ii,jj in zip(i[slices],j[slices])]
                bi = [int(ii) for ii in i[slices]]
                bj = [int(jj) for jj in j[slices]]
                c = iou_matrix([i[slices]],[j[slices]])[0][0]
                print(f'Frame {i[0]}:',f'{c:>5.3f}',a,bi,bj)
         
        if os.path.isfile(results_path):
            pred_bb = torch.tensor(load_text(str(results_path), delimiter=('\t', ','), dtype=np.float64))
        else:
            # print('Result not found. {}'.format(results_path))
            continue
        for frame_id, b in enumerate(obj_gt):
            if b[0] == -1: continue
            gt.append([frame_id+1,1,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
        for frame_id, b in enumerate(pred_bb):
            if b[0] == -1: continue
            pred.append([frame_id+1,1,b[0],b[1],b[2],b[3],1.,-1.,-1.,-1.])
            
        if seq.name in ['GOT-10k_Val_000007','GOT-10k_Val_000016','GOT-10k_Val_000021']: compare()
            
        if len(gt) == 0: continue
        accs.append(motMetricsEnhancedCalculator(np.array(gt, dtype=np.float64), np.array(pred, dtype=np.float64)))
        names.append(seq.name)
        
    mh = mm.metrics.create()
    summary = mh.compute_many(accs, metrics=['mota', 'motp', 'num_frames', 'num_objects' , 'idf1', 'idp', 'idr', \
                                        'recall', 'precision',  \
                                        'mostly_tracked', 'partially_tracked', \
                                        'mostly_lost', 'num_false_positives', \
                                        'num_misses', 'num_switches', \
                                        'num_fragmentations'
                                    ], \
                        generate_overall=True,
                        names=names)
    df =  pd.DataFrame.from_dict(summary)
    namemap = {'num_objects': 'GT', 'mota': 'MOTA', 'motp' : 'MOTP', \
                 'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', 'precision': 'Prcn',  \
                'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
                'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
                'num_misses': 'FN', 'num_switches' : 'IDsw', \
                'num_fragmentations' : 'FM', 'num_frames': 'Frames'
                }
    df.rename(columns=namemap, inplace=True)
    return df

# one_track_single(trackers[0])
len(d1),len(d2)

YouTubeVOS/2019/probio_train loaded.


YouTubeVOS: 100%|██████████| 63/63 [00:38<00:00,  1.63seq/s]


YouTubeVOS/2019/probio_valid loaded.


(100, 63)

In [4]:
# path = '/mnt/data/qizhezhang/pytracking/ltr/train_settings'
# ltr1 = '/mnt/data/qizhezhang/pytracking/ltr/ckpt/checkpoints/ltr'
# ltr2 = '/mnt/data/qizhezhang/pytracking/ltr/ckpt/tensorboard/ltr'
# for model in os.listdir(path):
#     if model == '__init__.py' or model == '__pycache__': continue
#     for param in os.listdir(os.path.join(path,model)):
#         if param == '__init__.py' or param == '__pycache__': continue
#         param = param.split('.')[0]
#         # if os.path.isdir(os.path.join(ltr1,model,param)): continue
#         # else:
#         #     if not os.path.isdir(os.path.join(ltr1,model)): os.mkdir(os.path.join(ltr1,model))
#         #     os.mkdir(os.path.join(ltr1,model,param))
#         if os.path.isdir(os.path.join(ltr2,model,param)): continue
#         else:
#             if not os.path.isdir(os.path.join(ltr2,model)): os.mkdir(os.path.join(ltr2,model))
#             os.mkdir(os.path.join(ltr2,model,param))

In [11]:
# import shutil
# net_path = '/mnt/data/qizhezhang/pytracking/pytracking/networks'
# name = ['prdimp18','prdimp50','super_dimp','super_dimp_simple']
# for name in name:
#     shutil.copy(os.path.join(net_path,name+'.pth.tar'),os.path.join(ltr1,'dimp',name,'DiMPnet_ep0000.pth'))

In [13]:
# import shutil
# net_path = '/mnt/data/qizhezhang/pytracking/pytracking/networks'
# name = ['prdimp18','prdimp50','super_dimp','super_dimp_simple','dimp18','dimp50']
# for name in name:
#     shutil.copy(os.path.join(ltr1,'dimp',name,'DiMPnet_ep0055.pth.tar'),os.path.join(net_path,name+'.pth.tar'))

In [2]:
import shutil,os
origin_path = '/mnt/data/qizhezhang/pytracking/pytracking/networks_old'
net_path = '/mnt/data/qizhezhang/pytracking/pytracking/networks_tune'
root_net = '/mnt/data/qizhezhang/pytracking/'
a = [
    # ('ltr/ckpt/checkpoints/ltr/kys/kys/KYSNet_ep0050.pth','kys.pth'),
    # ('ltr/ckpt/checkpoints/ltr/lwl/lwl_boxinit/LWTLBoxNet_ep0060.pth','lwl_boxinit.pth'),
    # ('ltr/ckpt/checkpoints/ltr/dimp/prdimp50/DiMPnet_ep0055.pth','prdimp50.pth.tar'),
    # ('ltr/ckpt/checkpoints/ltr/dimp/dimp50/DiMPnet_ep0060.pth','dimp50.pth'),
    # ('ltr/ckpt/checkpoints/ltr/bbreg/atom/ATOMnet_ep0050.pth','atom_default.pth'),
    # ('ltr/ckpt/checkpoints/ltr/tomp/tomp101/ToMPnet_ep0310.pth','tomp101.pth.tar'),
    ('ltr/ckpt/checkpoints/ltr/tamos/tamos_swin_base/TaMOsNet_ep0000.pth','tamos_swin_base.pth.tar'),
    ('ltr/ckpt/checkpoints/ltr/tamos/tamos_resnet50/TaMOsNet_ep0000.pth','tamos_resnet50.pth.tar')
    ]
# for i in a:
    # shutil.copy(os.path.join(root_net,i[0]),os.path.join(net_path,i[1]))
for i in a:
    shutil.copy(os.path.join(origin_path,i[1]),os.path.join(root_net,i[0]))